In [1]:
!nvidia-smi

Mon May 10 13:14:16 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.73.01    Driver Version: 460.73.01    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  GeForce RTX 208...  Off  | 00000000:01:00.0  On |                  N/A |
| 26%   34C    P8     9W / 260W |    589MiB / 11018MiB |      4%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
# !pip install transformers==4.1.0
# !pip install git+https://github.com/openai/CLIP.git

In [3]:
import torch
from torch import nn
from tqdm import tqdm_notebook
import torch.nn.functional as F
import numpy as np

from torch.utils.data import DataLoader, Dataset
from utils import seed_all
from data import split_dataset, CityFlowNLDataset, CityFlowNLDatasetInference
from model import SiameseBaselineModel
from learner import Learner
from ensemble import submit, process_test

In [4]:
seed_all(2010)

In [5]:
data_root = './extracted_data/'
json_path = './extracted_data/train-tracks.json'

test_tracks_file = './extracted_data/test-tracks.json'
test_query_file = './extracted_data/test-queries.json'

In [6]:
train_data, val_data = split_dataset(json_path)

In [7]:
train_ds = CityFlowNLDataset(data_root, train_data, k_neg = 5)
val_ds = CityFlowNLDatasetInference(data_root, val_data)

In [8]:
train_dl = DataLoader(train_ds, batch_size= 4, shuffle = True)
val_dl = DataLoader(val_ds, batch_size= 32, shuffle=  False)

In [9]:
model = SiameseBaselineModel(weight_lang_loss = 0.3, 
                             t = 0.07, 
                             use_clip = True, 
                             share_weights = True,
                             use_rnn = True, 
                             use_crop = True, 
                             is_triplet_loss = False)
model.cuda()
# model.float()
learner = Learner(epoch = 10, lr = 1e-5, accu_grad_step = 4,
                  train_dl = train_dl, val_dl = val_dl,
                  model = model)

100%|███████████████████████████████████████| 354M/354M [00:40<00:00, 8.83MiB/s]


In [10]:
learner.eval_one_epoch(val_dl)

(0.0171990171990172, 0.02702702702702703, 0.020747801787136748)

In [ ]:
# set name for model
model_name = 'test.pth'
learner.train(model_name, print_train = False, lr = 1e-5, epoch = 5)

/home/quangph3/Workspace/AICity2021-Track5/adam16.py:58: UserWarning: This overload of add_ is deprecated:
	add_(Number alpha, Tensor other)
Consider using one of the following signatures instead:
	add_(Tensor other, *, Number alpha) (Triggered internally at  /pytorch/torch/csrc/utils/python_arg_parser.cpp:882.)
  exp_avg.mul_(beta1).add_(1 - beta1, grad)
/home/quangph3/anaconda3/envs/eventext/lib/python3.8/site-packages/torch/nn/modules/rnn.py:581: UserWarning: RNN module weights are not part of single contiguous chunk of memory. This means they need to be compacted at every call, possibly greatly increasing memory usage. To compact weights again call flatten_parameters(). (Triggered internally at  /pytorch/aten/src/ATen/native/cudnn/RNN.cpp:775.)
  result = _VF.lstm(input, hx, self._flat_weights, self.bias, self.num_layers,



loss: 1.1236456341618866



Save model ....
epoch 0 : recall_5: 0.1597051597051597, recall_10 : 0.28501228501228504, mrr: 0.12374267459585975


In [18]:
# after having finished training all the models
# results is in submission form
results = submit(learner, './weights', test_query_file, test_tracks_file)